In [1]:
#Rodar o codigo inteiro, selecionando o mês a ser calculado abaixo:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from openpyxl import load_workbook
import olx_datalake

#Mês a ser calculado:
target_date = '2023-09-01'

#conversoes necessarias:
year_str, month_str, day_str = target_date.split('-')

# Convert year_str and month_str to integers
year = int(year_str)
month = int(month_str)

#chamada do datalake_olx
dl = olx_datalake.DataLake()

def column_index_from_string(column_str):
    """Converts a column name (e.g., 'A', 'AB', 'ZZ') to its numeric index."""
    base = ord('Z') - ord('A') + 1
    column_index = 0
    for char in column_str:
        if 'A' <= char <= 'Z':
            column_index = column_index * base + (ord(char) - ord('A') + 1)
        else:
            raise ValueError("Invalid column name: " + column_str)
    return column_index

def write_value_to_cell_on_date(value, tab_name, target_date, target_column):
    # Replace 'your_credentials.json' with the path to your Google Sheets credentials JSON file.
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('project-bht-e1805246c668.json', scope)
    
    # Replace 'your_spreadsheet' with the name of your Google Sheets spreadsheet.
    gc = gspread.authorize(credentials)
    worksheet = gc.open('Growth | Main Metrics').worksheet(tab_name)
    
    # Get all values from column A (date column)
    date_column = worksheet.col_values(1)
    
    # Find the row index with the matching target_date
    row_index = date_column.index(target_date) + 1

    # Convert the target_column name (e.g., 'AB') to numeric column index
    target_column_index = column_index_from_string(target_column.upper())

    # Write the value to the specified cell in the matching row and column
    worksheet.update_cell(row_index, target_column_index, value)
    
    print(f"Value {value} written to cell {target_date} in tab {tab_name}, row {row_index}, column {target_column}")
print("Tudo certo! Pode rodar as métricas!")

Tudo certo! Pode rodar as métricas!


In [ ]:
#Monthly average of daily active listers
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'N'  
metrica_name='monthly_avg_daily_active_listers'
#query da métrica
month_measure = dl.sql("""
    with
olx_calendar as (
SELECT 
date_trunc('day',event_date) as ref_day 
FROM ods.calendar 
where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
),

daily as (
select
    b.ref_day,
    count(distinct account_id_fk) as listers_ativos
    from olx_listing.ad_inventory as a
    join olx_calendar as b on date(start_date) <= b.ref_day
    and (end_date is null or end_date>= b.ref_day)
    join ods.ad as c on c.list_id_nk=a.list_id_nk
    and date_trunc('month',cast(array_join(array[year, month, day], '-') as date)) between date('"""+target_date+"""')- interval '10' month and date('"""+target_date+"""') 
    group by 1
    )

select
avg(listers_ativos) as """+metrica_name+""",
date_trunc('month',ref_day) as event_date
from daily
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

FINISHED: 100%|██████████| 2247/2247 [01:30<00:00, 24.84 splits/s]


In [ ]:
#Monthly average of daily active ads
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'J'  
metrica_name='monthly_avg_daily_active_ads'
#query da métrica
month_measure = dl.sql("""
    with
olx_calendar as (
SELECT 
date_trunc('day',event_date) as ref_day 
FROM ods.calendar 
where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
),

daily as (
select
    b.ref_day,
    count(distinct list_id_nk) as ads_ativos
    from olx_listing.ad_inventory as a
    join olx_calendar as b on date(start_date) <= b.ref_day
    and (end_date is null or end_date>= b.ref_day)
    group by 1
    )

select
avg(ads_ativos) as """+metrica_name+""",
date_trunc('month',ref_day) as event_date
from daily
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
ads_ativos=numeric_value 
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)


In [ ]:
#Listers
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'I'  
metrica_name='listers'
#query da métrica
month_measure = dl.sql("""
    SELECT 
    n_total_listers_email as """+metrica_name+""",
    event_date
FROM dw.lister_monthly
WHERE platform_name='(All)'
    AND platform_main_name='(All)'
    AND state_name= '(All)'
    AND ad_lister_type_name='(All)'
    AND ad_type_name='(All)'
    AND area_name='(All)'
    AND category_name='(All)'
    AND category_main_name='(All)'
    AND category_group_name='(All)'
    AND lister_type_name='(All)'
    AND date_trunc('month',event_date)= date('"""+target_date+"""')
    AND year="""+str(year)+"""
    AND month="""+str(month)+"""
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)




In [ ]:
#Lost Listers
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'K' 
metrica_name='lost_listers'
#query da métrica
month_measure = dl.sql("""
    with

listers_list as ( 
select 
distinct account_id_fk as user_id,
date_trunc('month',approval_date) as publi_month
from ods.ad
where date_trunc('month',cast(array_join(array[year, month, day], '-') as date)) in (date('"""+target_date+"""'),date('"""+target_date+"""')- interval '1' month,date('"""+target_date+"""')- interval '2' month)
and date_trunc('month',approval_date) in (date('"""+target_date+"""'),date('"""+target_date+"""')- interval '1' month)
group by 1,2
),

dated_list as ( 
select
user_id, 
publi_month,
date_trunc('month',publi_month + interval '1' month) as next_calendar_month,
lead(publi_month) over (partition by user_id order by publi_month) as next_publi_month
from listers_list
)

select
count(distinct user_id) as """+metrica_name+""",
publi_month as event_date
from dated_list
where (date_diff('month',publi_month,next_publi_month)>1 or next_publi_month is null)
and publi_month= date('"""+target_date+"""')
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)



In [ ]:
#Paying listers
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'L'  
metrica_name='paying_listers'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
    select
count( distinct user_id_fk) as """+metrica_name+""",
date_trunc('month',payment_event_date) as ref_month
from olx_payment.metric_monetization_daily  
where True
--and category_name='Carros, vans e utilitários'
--and user_type not like '%Active%'
and status='PAID' 
and date_trunc('month',payment_event_date) =date('"""+target_date+"""')
--and product_channel='payg' 
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#NNL
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'M'  
metrica_name='nnl'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
    select 
count(distinct list_id_nk) as """+metrica_name+""",
date_trunc('month', approval_date) as ref_month
from ods.ad
where True
and date_trunc('month',cast(array_join(array[year, month, day], '-') as date)) in (date('"""+target_date+"""'),date('"""+target_date+"""') - interval '1' month, date('"""+target_date+"""') - interval '2' month)
and date_trunc('month',approval_date) =date('"""+target_date+"""')
and approval_date is not null
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Leads
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'P'  
metrica_name='leads'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
    select 
count(*) as """+metrica_name+""",
date_trunc('month',real_date) as event_date
from ods.ad_contacts as b
where True
and date_trunc('month',cast(array_join(array[b.year, b.month, b.day], '-') as date)) =date('"""+target_date+"""')
and date_trunc('month',real_date) =date('"""+target_date+"""') 
and (buyer_account_id_fk != seller_account_id_fk or buyer_account_id_fk is null or buyer_account_id_fk = -1) --excluir seller fazendo adview no seu anuncio
and message_rank=1 --primeiro contato
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Buyers
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'Q'  
metrica_name='buyers'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
    select 
count(distinct case when (b.buyer_account_id_fk is null or b.buyer_account_id_fk=-1) then id_tracker
else cast(b.buyer_account_id_fk as varchar) end) as """+metrica_name+""",
date_trunc('month',real_date) as event_date
from ods.ad_contacts as b
where True
and date_trunc('month',cast(array_join(array[b.year, b.month, b.day], '-') as date)) =date('"""+target_date+"""')
and date_trunc('month',real_date) =date('"""+target_date+"""') 
and (buyer_account_id_fk != seller_account_id_fk or buyer_account_id_fk is null or buyer_account_id_fk = -1) --excluir seller fazendo adview no seu anuncio
and message_rank=1 --primeiro contato
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Unique Leads
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'S' 
metrica_name='unique_leads'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with base as(
select 
count(distinct list_id_nk) as n_leads,
case when (b.buyer_account_id_fk is null or b.buyer_account_id_fk=-1) then id_tracker
else cast(b.buyer_account_id_fk as varchar) end as buyer_account_id_fk,
date_trunc('month',real_date) as event_month

from ods.ad_contacts as b
where True
and date_trunc('month',cast(array_join(array[b.year, b.month, b.day], '-') as date)) =date('"""+target_date+"""')
and date_trunc('month',real_date) =date('"""+target_date+"""') 
and (buyer_account_id_fk != seller_account_id_fk or buyer_account_id_fk is null or buyer_account_id_fk = -1) --excluir seller fazendo adview no seu anuncio
and message_rank=1 --primeiro contato
group by 2,3
)
select 
sum(n_leads) as """+metrica_name+""",
event_month as event_date
from base
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
# % of ads with view
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'V'  
metrica_name='active_ads_with_view'
#query da métrica
aux_measure = dl.sql("""
    with
olx_calendar as (
SELECT 
date_trunc('day',event_date) as ref_day 
FROM ods.calendar 
where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
),

daily as (
select
    b.ref_day,
    count(distinct list_id_nk) as ads_ativos
    from olx_listing.ad_inventory as a
    join olx_calendar as b on date(start_date) <= b.ref_day
    and (end_date is null or end_date>= b.ref_day)
    group by 1
    )

select
avg(ads_ativos) as """+metrica_name+""",
date_trunc('month',ref_day) as event_date
from daily
group by 2
""")
#medida auxiliar
numeric_value = float(aux_measure.iloc[0][metrica_name])
ads_ativos=numeric_value 

month_measure = dl.sql("""
    with

daily as (
select 
                count(distinct b.list_id_nk) as n_ads, 
                b.real_date as event_date 
            from ods.traffic_info as b
            join ods.ad as a on b.list_id_nk=a.list_id_nk
            where True
            and date_trunc('month',cast(array_join(array[b.year, b.month, b.day], '-') as date)) = date('"""+str(target_date)+"""') 
            and date_trunc('month',real_date) = date('"""+str(target_date)+"""') 
            and coalesce( b.account_id_fk, -1) != a.account_id_fk --excluir seller fazendo adview no seu anuncio
            and b.adview=1 --flag indicando que essa linha é um adview,pegando somente linhas com adviews
            group by 2
    )

select
avg(n_ads) as """+metrica_name+""",
date_trunc('month',event_date) as event_date
from daily
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(1.00*numeric_value/ads_ativos, sheet_tab_name, target_date, target_column)


In [ ]:
#Unique Buyers per lister
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'X' 
metrica_name='unique_buyers_per_lister'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with base as (
            select 
count(distinct case when (b.buyer_account_id_fk is null or b.buyer_account_id_fk=-1) then id_tracker
else cast(b.buyer_account_id_fk as varchar) end) as n_buyers,
real_date as daily_date,
seller_account_id_fk


from ods.ad_contacts as b
where date_trunc('month',cast(array_join(array[b.year, b.month, b.day], '-') as date)) = date('"""+str(target_date)+"""') 
and date_trunc('month',real_date) = date('"""+str(target_date)+"""') 
and (buyer_account_id_fk != seller_account_id_fk or buyer_account_id_fk is null or buyer_account_id_fk = -1) --excluir seller fazendo adview no seu anuncio

and message_rank=1 --primeiro contato
group by 2,3
),

aux as (
select
avg(n_buyers) as bpl,
daily_date
from base
group by 2
)

select
avg(bpl) as """+metrica_name+""",
date_trunc('month',daily_date) as event_date
from aux
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Number of buyers with transaction
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'Y' 
metrica_name='buyers_with_transaction'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
SELECT
count( distinct buyer_account_id_fk)  as """+metrica_name+""",
date_trunc('month',event_date) as event_date 

FROM ods.olxpay_payments 
where True
and date_trunc('month',try_cast(array_join(array[year, month, day], '-') as date)) = date('"""+str(target_date)+"""') 
and date_trunc('month',event_date) = date('"""+str(target_date)+"""') 

and status in ('COMPLETED')
and source = 'CHAT'
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
bwt=numeric_value
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#GMV-Compra segura
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'Z' 
metrica_name='gmv_olxpay'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
SELECT
sum(total_value) as """+metrica_name+""",
date_trunc('month',event_date) as event_date

FROM ods.olxpay_payments
where 1=1
and date_trunc('month',try_cast(array_join(array[year, month, day], '-') as date)) = date('"""+str(target_date)+"""') 
and status in ('COMPLETED')
and source = 'CHAT'
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Total transactions TXN-Compra segura
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'AB' 
metrica_name='total_olxpay_transactions'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""

SELECT
count( distinct transfer_id_nk) as """+metrica_name+""",
date_trunc('month',event_date) as event_date 

FROM ods.olxpay_payments 
where True
and date_trunc('month',try_cast(array_join(array[year, month, day], '-') as date)) = date('"""+str(target_date)+"""') 
and date_trunc('month',event_date) = date('"""+str(target_date)+"""') 
and status in ('COMPLETED')
and source = 'CHAT'
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
total_transactions=numeric_value
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Frequency of purchase with transaction
#IMPORTANTE: Rodar as metricas "Number of buyers with transaction" e "Total transactions TXN-Compra segura"antes dessa
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'AA' 
metrica_name='frequency_of_purchase'
numeric_value=1.00*total_transactions/bwt
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Adoption rate transaction
# Parametros da métrica:
sheet_tab_name = 'Histórico OLX'
target_column = 'AD' 
metrica_name='adoption_rate_transactions'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
sold_measure = dl.sql("""
select 

count( distinct a.list_id_nk) as """+metrica_name+""",
date_trunc('month',a.event_date) as event_date

from ods.ad_actions as a
join ods.dm_reason_removed_detail as b on (case when b.reason_removed_detail_id_nk='NA' then -1 else cast(b.reason_removed_detail_id_nk as bigint) end ) =a.reason_removed_detail_id_fk
join ods.ad as c on a.ad_id_fk=c.ad_id_pk
where 1=1
and upper(b.reason_removed_detail_name) like '%OLX%'
and action_type='delete'
and date_trunc('month',try_cast(array_join(array[a.year, a.month, a.day], '-') as date)) = date('"""+str(target_date)+"""') 

group by 2

""")
#medida no mês
sold_via_olx = float(sold_measure.iloc[0][metrica_name])
numeric_value=total_transactions/sold_via_olx
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
###ZAP###

In [ ]:
#listers
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'I'  
metrica_name='listers'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
select 
count(distinct advertiser_id) as """+metrica_name+""",
date_trunc('month',created_at) as event_date

from  silver_normalized.listings_snapshots_daily 
where True
and status in ('ACTIVE', 'DELETED','INACTIVE')
and dt = current_date - interval '1' day
and date_trunc('month',created_at) =date('"""+target_date+"""')

group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Monthly average of daily active ads
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'J'  
metrica_name='monthly_avg_active_ads'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with

ativos_diarios as (
select 
count(distinct id) as ativos,
dt

from silver_normalized.listings_snapshots_daily 
where True
and status in ('ACTIVE')
and date_trunc('month',dt) = date('"""+target_date+"""')
group by 2
)

select
avg(ativos) as """+metrica_name+""",
date_trunc('month',dt) as event_date
from ativos_diarios
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
active_ads_zap=numeric_value
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Lost Listers
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'K'  
metrica_name='lost_listers'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with

listers_list as ( 
select 
distinct advertiser_id as user_id,
date_trunc('month',created_at) as publi_month
from silver_normalized.listings_snapshots_daily
where dt = current_date - interval '1' day
and status in ('ACTIVE', 'DELETED','INACTIVE')
and date_trunc('month',created_at) between date('"""+target_date+"""')- interval '5' month and date('"""+target_date+"""')
group by 1,2
),

dated_list as ( 
select
user_id, 
publi_month,
date_trunc('month',publi_month + interval '1' month) as next_calendar_month,
lead(publi_month) over (partition by user_id order by publi_month) as next_publi_month
from listers_list
)

select
count(distinct user_id) as """+metrica_name+""",
publi_month as event_date
from dated_list
where (date_diff('month',publi_month,next_publi_month)>1 or next_publi_month is null)
and next_calendar_month < date_trunc('month',current_date)
and publi_month=date('"""+target_date+"""') - interval '1' month
group by 2


""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#NNL
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'L'  
metrica_name='nnl'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
select 
count(distinct id)  as """+metrica_name+""",
date_trunc('month',created_at) as event_date

from silver_normalized.listings_snapshots_daily 
where True
and status in ('ACTIVE', 'DELETED','INACTIVE')
and dt = current_date - interval '1' day
and date_trunc('month',created_at) = date('"""+target_date+"""')
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Monthly average of daily active listers
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'M'  
metrica_name='monthly_avg_active_listers'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with

ativos_diarios as (
select 
count(distinct advertiser_id) as ativos,
dt

from silver_normalized.listings_snapshots_daily 
where True
and status in ('ACTIVE')
and date_trunc('month',dt) =date('"""+target_date+"""')
group by 2
)

select
avg(ativos) as """+metrica_name+""",
date_trunc('month',dt) as event_date
from ativos_diarios
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Buyers
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'P'  
metrica_name='buyers'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
select 
count(distinct case when (accountid is null ) then fingerprint
else accountid end) as """+metrica_name+""",
date_trunc('month',dt) as event_date

from  communication.valid_leads
where True
and date_trunc('month',dt) =date('"""+target_date+"""')
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Leads
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'O'  
metrica_name='leads'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
select 
count(*) as """+metrica_name+""",
date_trunc('month',dt) as event_date

from  communication.valid_leads
where True
and date_trunc('month',dt) =date('"""+target_date+"""')
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Unique Leads
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'R'  
metrica_name='unique_leads'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with

buyer_per_list as (
select 
count( distinct case when (accountid is null ) then fingerprint
else accountid end) as bpl,
listingid as list_id,
date_trunc('month',dt) as lead_month

from  communication.valid_leads
where True

and date_trunc('month',dt) =date('"""+target_date+"""')

group by 2,3
)

select
sum(bpl) as """+metrica_name+""",
lead_month as event_date
from buyer_per_list
group by 2
""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
# % ads with view
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'S'  
metrica_name='ads_with_view'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with

daily as (
select 
count(distinct attributes.listing_id) as n_ads_with_view,
dt
FROM  data_platform.clickstream_listing_rendered_event 
where date_trunc('month',dt) =date('"""+target_date+"""')
and name='ListingRendered'
group by 2
)

select
avg(n_ads_with_view) as """+metrica_name+""",
date_trunc('month',dt) as event_date
from daily
group by 2

""")
#medida no mês
awv = float(month_measure.iloc[0][metrica_name])
numeric_value=1.00*awv/active_ads_zap
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)

In [ ]:
#Unique buyers per lister
# Parametros da métrica:
sheet_tab_name = 'Histórico ZAP+VR'
target_column = 'U'  
metrica_name='unique_buyers_per_lister'
#query da métrica
#where date_trunc('month',date(event_date)) =date('"""+target_date+"""')
#as """+metrica_name+""",
month_measure = dl.sql("""
with

buyer_per_lister as (
select 
count(distinct case when (accountid is null ) then fingerprint
else accountid end) as buyers,
dt,
advertiserid,
date_trunc('month',dt) as lead_month
from  communication.valid_leads
where True
and date_trunc('month',dt) =date('"""+target_date+"""')
group by 2,3,4
),

daily_mean as (
select
avg(buyers) as avg_bpl_daily,
dt,
lead_month
from buyer_per_lister 
group by 2,3
)

select
avg(avg_bpl_daily)  as """+metrica_name+""",
lead_month as event_date
from daily_mean
group by 2

""")
#medida no mês
numeric_value = float(month_measure.iloc[0][metrica_name])
#escrever métrica na célula
write_value_to_cell_on_date(numeric_value, sheet_tab_name, target_date, target_column)